In [1]:
import os

os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib_functions
from typing import List, Optional
matplotlib_functions.setMatplotlibParam()
plt.viridis()
import joblib
import pickle
from include.generators import Generator
from saver import Saver
from include.callbacks import get_modelcheckpoint, ExtraValidation

import model_eval
from Data import Data_train, Data_eval
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import models.reg as rm

2023-02-28 19:07:18.562796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1920x1440 with 0 Axes>

In [2]:
general_save_dir = "/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/"
project_name = "test_to_delete"

In [3]:
path_train_nc = "/libre/dumontj/coco2/dl-input/2km_forBox/train_dataset.nc"
path_valid_nc = "/libre/dumontj/coco2/dl-input/2km_forBox/valid_dataset.nc"
path_extra_valid_nc = "/libre/dumontj/coco2/dl-input/2km_Box/valid_dataset.nc"

save_dir = os.path.join(general_save_dir, "box")

In [4]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2")
data.prepare_output_inversion()

2023-02-28 19:07:26.200082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 19:07:26.714067: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14482 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


data.x.train.shape (47160, 64, 64, 1)


In [5]:
exp_name = "xco2_prec_winds_noisy_boolperfseg_modelX"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [6]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [7]:
reg_builder = rm.Reg_model_builder(
    "essential",
    data.x.fields_input_shape,
    data.y.classes,
    data.x.n_layer,
    data.x.xco2_noisy_chans,
)
model = reg_builder.get_model()
model.compile("adam", "MeanAbsoluteError", metrics=tf.keras.losses.MeanAbsolutePercentageError())

In [8]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=2,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/2


2023-02-28 19:07:37.011947: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/model/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-28 19:07:37.748690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-28 19:07:38.308431: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-28 19:07:38.663328: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x556c89c48070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-28 19:07:38.663390: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-28 19:07:38.674049: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set 

1474/1474 [==============================] - ETA: 0s - loss: 8.3798 - mean_absolute_percentage_error: 45.8378
Epoch 1: val_loss improved from inf to 7.77569, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/box/xco2_prec_winds_noisy_boolperfseg_modelX/w_best.h5
extra_val_loss: [6.094306468963623, 30.839372634887695]
1474/1474 [==============================] - 26s 15ms/step - loss: 8.3798 - mean_absolute_percentage_error: 45.8378 - val_loss: 7.7757 - val_mean_absolute_percentage_error: 38.5127
Epoch 2/2
1473/1474 [============================>.] - ETA: 0s - loss: 7.6469 - mean_absolute_percentage_error: 42.1079
Epoch 2: val_loss improved from 7.77569 to 7.48140, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/box/xco2_prec_winds_noisy_boolperfseg_modelX/w_best.h5
extra_val_loss: [5.344578742980957, 27.591793060302734]
1474/1474 [==============================] - 21s 14ms/step - loss: 7.6458 - mean_absolute_percentage_error: 42.1047 - val_loss: 7.4

In [9]:
u = [5.06552791595459, 26.34125518798828]

In [10]:
if type(u) == list:
    print(True)

True


In [2]:
run main.py +experiment=inv_lip

2023-02-28 19:09:59.976649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/cerea_raid/users/dumontj/dev/coco2/dl


/cerea_raid/users/dumontj/dev/coco2/dl/main.py:18: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="cfg", config_name="config")
/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(



 
 
 
 
 Run begins 
 
 
 
 

data:
  split:
    type: regular
    train:
      ratio: 0.85
  input:
    dir_seg_models: /cerea_raid/users/dumontj/dev/coco2/dl/res/models
    chan_0: xco2
    chan_1: xco2_prec
    chan_2: v_wind
    chan_3: u_wind
    chan_4: bool_perf_seg
  output:
    N_emissions: 1
  path:
    directory: /libre/dumontj/coco2/dl-input
    train:
      name: 2km_forLip_2
      nc: train_dataset.nc
    valid:
      name: 2km_Lip
      nc: valid_dataset.nc
    extra_valid:
      name: 2km_forLip_2
      nc: valid_dataset.nc
dir_res: res/inversion
exp_name: inv_lip_perf_seg
seed: 42
sweep: true
model:
  type: inversion
  name: essential
  loss_func: MeanAbsoluteError
  dropout_rate: 0.2
  scaling_coefficient: 1
training:
  batch_size: 32
  learning_rate: 0.001
  max_epochs: 500
  init_weights: random
  optimiser: adam
augmentations:
  shuffle: true
  rot:
    range: 180
  shift:
    range: 0
  flip:
    bool: true
  shear:
    range: 90
  zoom:
    range: 0.2
callbacks:

wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


...

wandb: Adding directory to artifact (/cerea_raid/users/dumontj/dev/coco2/dl/res/inversion/inv_lip_perf_seg/wandb/run-20230228_191010-7bva2nrq/files/model-best)... Done. 0.1s


extra_val_loss: [5.558135032653809, 50.49557113647461]
786/786 [==============================] - 16s 20ms/step - loss: 7.3407 - mean_absolute_percentage_error: 33.0828 - val_loss: 3.7361 - val_mean_absolute_percentage_error: 25.8615 - lr: 0.0010
Epoch 10/500
783/786 [============================>.] - ETA: 0s - loss: 7.3057 - mean_absolute_percentage_error: 32.5539
Epoch 10: val_loss did not improve from 3.73610
extra_val_loss: [5.066624641418457, 42.77330017089844]
786/786 [==============================] - 13s 16ms/step - loss: 7.3018 - mean_absolute_percentage_error: 32.5373 - val_loss: 4.5006 - val_mean_absolute_percentage_error: 30.6445 - lr: 0.0010
Epoch 11/500
785/786 [============================>.] - ETA: 0s - loss: 7.1490 - mean_absolute_percentage_error: 31.8070
Epoch 11: val_loss did not improve from 3.73610
extra_val_loss: [4.689034938812256, 35.55213928222656]
786/786 [==============================] - 13s 16ms/step - loss: 7.1504 - mean_absolute_percentage_error: 31.8068

wandb: Adding directory to artifact (/cerea_raid/users/dumontj/dev/coco2/dl/res/inversion/inv_lip_perf_seg/wandb/run-20230228_191010-7bva2nrq/files/model-best)... Done. 0.1s


extra_val_loss: [4.679994583129883, 42.482913970947266]
786/786 [==============================] - 15s 19ms/step - loss: 6.8821 - mean_absolute_percentage_error: 30.6709 - val_loss: 3.7298 - val_mean_absolute_percentage_error: 25.6895 - lr: 0.0010
Epoch 17/500
786/786 [==============================] - ETA: 0s - loss: 6.7552 - mean_absolute_percentage_error: 30.5718
Epoch 17: val_loss did not improve from 3.72977
extra_val_loss: [5.2524919509887695, 43.63689041137695]
786/786 [==============================] - 13s 16ms/step - loss: 6.7552 - mean_absolute_percentage_error: 30.5718 - val_loss: 4.2815 - val_mean_absolute_percentage_error: 30.0090 - lr: 0.0010
Epoch 18/500
785/786 [============================>.] - ETA: 0s - loss: 6.7520 - mean_absolute_percentage_error: 30.1210
Epoch 18: val_loss did not improve from 3.72977
extra_val_loss: [4.475597381591797, 37.757843017578125]
786/786 [==============================] - 13s 16ms/step - loss: 6.7517 - mean_absolute_percentage_error: 30.1

KeyboardInterrupt: 